# ASTR3110 Tutorial 4: MCMC

Tutorial 4 of the *'Data Science Techniques in Astrophysics'* course at Macquarie University.

## Learning outcomes from this tutorial

 * Understand what a sampler achieves
 * Understand the terms in Bayes's Formula and how they relate to fitting models
 * Use a MCMC sampler to fit a polynomial model to a simple 1D spectrum
 * Create a triangle plot to show correlations between parameters
 * Extract best-fit and uncertainty estimates from an MCMC chain


## Likelihood, Priors and Bayes's Theorem

In Tutorial 3 we used $\chi^2_{\rm model}$ difference between the model and data as a measure of *goodness of fit* and found the best-fitting model by minimizing its value. However, we often have *prior* independent measurements of the parameters we want to fit. We want this prior information to weight the total $\chi^2$ value so that it prefers parameter values closer to our previous measurements. Each prior parameter estimate contributes its own $\chi^2$ value. For example, for a parameter $p = p_{\rm prior} \pm \sigma_{\rm prior}$, the $\chi^2_{\rm prior}$ value is given by:

$$\chi^2_{\rm prior} = \left(\frac{p_{\rm model} - p_{\rm prior}}{\sigma_{\rm prior}}\right)^2$$.

The total $\chi^2$ value is then given by the sum of all values: $\chi^2 = \chi^2_{\rm model} + \chi^2_{\rm prior}$.

The *likelihood* is a related way of assessing a model fit: *what is the probability, or likelhood $\mathcal{L}$, of getting the observed data given particular values of parameters p?* For Gaussian distributed likelihoods the likelihood is related to $\chi^2$ via

$$\mathcal{L} = exp(-\chi^2\big/2)~~~~or~~~~-2~{\rm ln}(\mathcal{L}) = \chi^2,$$

so that $\chi^2$ is often referred to as the 'log-likelihood'. Because $\mathcal{L}$ is in log-space, likelihoods are *multiplied* to get the total $\mathcal{L}$. From a numerical perspective, the log-likelihood is also a smaller number and less likely to run into computational limits.

### Bayes's Theorem

For a model hypothesis $\mathcal{M}$
and data $\boldsymbol{d}$, the posterior probability for a set of
model parameters $\boldsymbol{\theta}$ is given by Bayes' Theorem
\begin{equation}\label{eqn:bayes_formula}
  {\rm Pr}(\boldsymbol{\theta}|\boldsymbol{d}, \mathcal{M}) =
  \frac{{\rm Pr}(\boldsymbol{d}|\boldsymbol{\theta},
    \mathcal{M})\,{\rm Pr}(\boldsymbol{\theta}|\mathcal{M})}
       {{\rm Pr}(\boldsymbol{d}|\mathcal{M})}.
\end{equation}
The term ${\rm Pr}(\boldsymbol{d}|\boldsymbol{\theta}, \mathcal{M})$
is the probability of the data given the model parameters, also known
as the likelihood $\mathcal{L}$.  The term ${\rm Pr}(\boldsymbol{\theta}|\mathcal{M})$ is known as the
prior; it encodes the probability of the
current parameter values given the proposed model, i.e., any  *a
  priori* information about the parameter values. The final term ${\rm Pr}(\boldsymbol{d}|\mathcal{M})$, known as the
evidence, is the probability of the data given the model and serves to
normalise the posterior distribution so that the total probability is
unity.

I found the discussion by Adrian Price-Whelan of the Flatiron Institute to be very helpful for understanding the above: [see this link](https://adrian.pw/blog/fitting-a-line/).

## The MCMC Sampler

## Setup for Google Drive and data access

As with the tutorial last week, we will be operating on actual data, so please mount your Google Drive:

In [1]:
# Comment out if running locally
#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
# Get into the correct directory (replace with the location of your copy)
#cd gdrive/"My Drive"/ASTR3110_Tutorial_Notebooks_2021 

We will be processing the same polarised radio-wavelength spectrum as last week. If you haven't already downloaded it, you can get it from this link: [HotSpot.csv](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2022/blob/master/DATA/HotSpot.csv). Save this file to your Google Drive under, for example, your 'DATA/' directory.

As a reminder, the file contains 7 columns corresponding to:

[frequency_Hz, I_mJy, Q_mJy, U_mJy, dI_mJy, dQ_mJy, dU_mJy]

We will want to access the columns frequency_Hz, I_mJy (flux in milly Janskys) and dI_mJy (uncertainty in flux). Start by reading all of the file into a Pandas dataframe:

In [3]:
#Same as Tutorial 3
import pandas as pd

# Read all columns into a Pandas dataframe
specDF  = pd.read_csv("DATA/HotSpot.csv")

# Convert Hz to GHz and add column
specDF['frequency_GHz'] = specDF['frequency_Hz']/1e9

print("Array has shape (columns, rows):", specDF.shape)

('Array has shape (columns, rows):', (7, 198))


As a shortcut, some useful functions are in the file ```Imports/util_tute04.py```. These can be imported into the current notebook like this:

In [4]:
# Import the function to evaluate a polynomial
from Imports.util_tute04 import polyN

# Import the function to plot the spectrum
from Imports.util_tute04 import plot_spec_polyN

You can examine *code* by using double question-marks to summon help:

In [5]:
# Examine the code for the imported function (show 1 ?, which gives short description vs ??, which gives code.)
#polyN?
#polyN??
#plot_spec_polyN??

You can see that this is our convenience function from Tutorial 3 for plotting the polynomial data and (optionally) best-fitting model. Use it now to check the data has been read in correctly:

In [6]:
%matplotlib inline

# Set the data arrays from the dataframe
xData = specDF['frequency_GHz'].values
yData = specDF['I_mJy'].values
dyData = specDF['dI_mJy'].values

# Plot the spectrum
plot_spec_polyN(xData, yData, dyData)

## Using MCMC to fit a Polynomial

Here we use the MCMC module ```emcee``` to fit our familiar spectrum using a polynomial model. Using a MCMC sampler has some distinct advantages when it comes to estimating the best-fitting parameters and uncertainty values.

Like before we define a *goodness-of-fit* function. However, this time we formulate it as a log-likelihood:

Unlike a gradient-decent method, we are not trying to go directly to a minimum point. Instead we want our guesses to spread out over parameter space *near the best-fit value*. Because the next guess of the chain is weighted by the likelihood, the 'walkers' spend most time near the peak likelihood and less time away from it. The density of samples can be used as a proxy for the posterior likelihood distribution.

For this excercise we will use a module called [emcee](https://emcee.readthedocs.io/en/stable/). This is a pure *Python* implementation of an MCMC sampler that makes many guesses in parallel. You can visualise each guess as a time-series of positions in parameter space that 'walk' all over parameter space - hence we call them 'walkers'.

Now that we have set our sampler parameters we can set it off walking to the answer. We have made no attempt at making a good initial guess, so our walkers will probably take a while to get close to the best fitting region. This is called the burn-in period and you need to visualise the history of each walker to see when they reach the right area of parameter space.

The sampler returns three things:

 * pos =   the last position of each sampler
 * prob =  the last likelihood for each sampler
 * state = the current state of the random number generator
 
The sampler also stores the history of each walker internally - refered to as a chain. We need to visualise the chains for each parameter to see how long it takes *all parameters* to burn-in. Let's loop over the parameters and look at how the chains evolve:

There is a nice version of this function in the file ```Imports/util_tute04.py```, so you can use that in future. This function plots the samples as points, coloured by likelihood.

Depending on your initial values, most chains have settled down to a flat (but finitely thick) trace. If they have not then you should run the burn-in for more steps.

Now we want to run the sampler again, but initializing each walker with the converged values, that is, the values on the extreme right of the plot above. These are stored in the ```pos``` variable from the last run of the sampler.

**Key Point:** 

Looking at a single plot above (e.g., the P1 parameter), we see that the samples are scattered around a single value of ~0.51. A histogram of sampled values is proportional to the *posterior likelihood distribution* of the P1 parameter.

Start by accessing a *flattened* version of the chain: all 100 parallel walker chains merged into one 1D array. Then plot the histogram of the values.

Now we can make estimates of the best-fit value and uncertainties using the method described in Tutorial 1.

We can also visualise how each parameter is correlated with every other parameter. Let's try checking how the polynomial coefficent P6 is correlated with P1:

Here we can see the P6 is anti-correlated with P1: solutions with higher values of P6 drive the P1 value down. 

Rather than making these plots by hand, there is a module called ```corner``` that is dedicated to plotting the results of MCMC samplers and similar. 

At this point we should calculate the best-fitting results and plot the model over the data as a visual check.